<a href="https://colab.research.google.com/github/akiran703/MultiInputPatientApproach/blob/master/reverse4input_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow.keras as tf
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten,Concatenate, Dropout
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.initializers import glorot_uniform
import os
import pandas as pd
import csv
import cv2
import random
import glob
from IPython.core.debugger import Tracer


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth 
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
desired_size=224
inputparameter = 4

In [ ]:
def identity_block(X, f, filters, stage, block):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters

    X_shortcut = X
   
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X = Add()([X, X_shortcut])# SKIP Connection
    X = Activation('relu')(X)

    return X

In [ ]:
def convolutional_block(X, f, filters, stage, block, s=2):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [ ]:
download_transfer = drive.CreateFile({'id':'1NTcd09yqjCh5a3kkWqiCGcclv8Nwo63q'})
download_transfer.GetContentFile('archive.zip')

In [ ]:
!unzip /content/archive.zip -d /content/transfer

Archive:  /content/archive.zip
  inflating: /content/transfer/New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132320.png  
  inflating: /content/transfer/New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132449.png  
  inflating: /content/transfer/New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132516.png  
  inflating: /content/transfer/New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132540.png  
  inflating: /content/transfer/New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132608.png  
  inflating: /content/transfer/New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132636.png  
  inflating: /content/transfer/New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132704.png  
  inflating: /content/transfer/New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132740.png  
  inflating: /content/transfer/New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132814.png  
  inflating: /content/transfer/New_Data_CoV2/Covid/Patient (10)/Anota

In [ ]:
covidtransfer = {}
for i in range(inputparameter):
  covidtransfer['pic'+str(i)] = []

healthytransfer = {}
for i in range(inputparameter):
  healthytransfer['pic'+str(i)] = []

In [ ]:
def transfernormalize(name):
    temptwo = []
    for i in range(len(name)):
        img = cv2.imread(name[i])
        # print(pId + " iam here" + f1)
        # imgreszie = cv2.resize(img, (480, 480))
        old_size = img.shape[:2]
        ratio = float(desired_size) / max(old_size)
        new_size = tuple([int(x * ratio) for x in old_size])
        img = cv2.resize(img, (new_size[1], new_size[0]))
        delta_w = desired_size - new_size[1]
        delta_h = desired_size - new_size[0]
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)
        color = [0, 0, 0]
        imgreszie = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
        normresize = (imgreszie* 1.0)/np.max(imgreszie)
        temptwo.append(normresize)
    transfersize(temptwo)

def nontransfernormalize(name):
    temptwo = []
    for i in range(len(name)):
        img = cv2.imread(name[i])
        # print(pId + " iam here" + f1)
        # imgreszie = cv2.resize(img, (480, 480))
        old_size = img.shape[:2]
        ratio = float(desired_size) / max(old_size)
        new_size = tuple([int(x * ratio) for x in old_size])
        img = cv2.resize(img, (new_size[1], new_size[0]))
        delta_w = desired_size - new_size[1]
        delta_h = desired_size - new_size[0]
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)
        color = [0, 0, 0]
        imgreszie = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
        normresize = (imgreszie* 1.0)/np.max(imgreszie)
        temptwo.append(normresize)
    healthytransfersize(temptwo)

In [ ]:
def transfersize(p):
    if(len(p) > inputparameter):
        while(len(p) != inputparameter):
            popvalue = random.randrange(len(p))
            p.pop(popvalue)
    elif(len(p) < inputparameter):
        while(len(p) != inputparameter):
            p.append(np.zeros((desired_size, desired_size, 3)))
    for i in range(len(p)):
        covidtransfer['pic' + str(i)].append(p[i])

def healthytransfersize(p):
    if(len(p) > inputparameter):
        while(len(p) != inputparameter):
            popvalue = random.randrange(len(p))
            p.pop(popvalue)
    elif(len(p) < inputparameter):
        while(len(p) != inputparameter):
            p.append(np.zeros((desired_size, desired_size, 3)))
    for i in range(len(p)):
        healthytransfer['pic' + str(i)].append(p[i])

In [ ]:
img_dir_transfer = "/content/transfer/New_Data_CoV2/Covid"
img_dirnon_transfer = "/content/transfer/New_Data_CoV2/Healthy"

healthy_patient_dirs = glob.glob("/content/transfer/New_Data_CoV2/Healthy/*/")
covid_patient_dirs = glob.glob("/content/transfer/New_Data_CoV2/Covid/*/")

# read in on an image-by-image basis
healthy_patient_imgs = glob.glob("/content/transfer/New_Data_CoV2/Healthy/*/*.png")
covid_patient_imgs = glob.glob("/content/transfer/New_Data_CoV2/Covid/*/*.png")

data=[]

labels=[]
healthylabels = []

for curr_dir in healthy_patient_dirs:
    list_of_imgs =[]
    imgs = glob.glob(curr_dir+'/*.png')
    for img in imgs:
        list_of_imgs.append(img)
    transfernormalize(list_of_imgs)
    labels.append([0])

for curr_dir in covid_patient_dirs:
    list_of_imgs =[]
    imgs = glob.glob(curr_dir+'/*.png')
    for img in imgs:
        list_of_imgs.append(img)
    nontransfernormalize(list_of_imgs)
    healthylabels.append([1])

In [ ]:
pic_1 = (list(covidtransfer.values())[0] + list(healthytransfer.values())[0])
pic_2 = (list(covidtransfer.values())[1] + list(healthytransfer.values())[1])
pic_3 = (list(covidtransfer.values())[2] + list(healthytransfer.values())[2])
pic_4 = (list(covidtransfer.values())[3] + list(healthytransfer.values())[3])

In [ ]:
print(np.shape(pic_1))
print(np.shape(pic_2))
print(np.shape(pic_3))
print(np.shape(pic_4))

(130, 224, 224, 3)
(130, 224, 224, 3)
(130, 224, 224, 3)
(130, 224, 224, 3)


In [ ]:
rand_order = np.random.permutation(len(pic_1))
pic_1 =np.array(pic_1)
pic_1 = pic_1[rand_order, :, :, :]

In [ ]:
rand_order = np.random.permutation(len(pic_2))
pic_2 =np.array(pic_2)
pic_2 = pic_2[rand_order, :, :, :]

In [ ]:
rand_order = np.random.permutation(len(pic_3))
pic_3 =np.array(pic_3)
pic_3 = pic_3[rand_order, :, :, :]

In [ ]:
rand_order = np.random.permutation(len(pic_4))
pic_4 =np.array(pic_4)
pic_4 = pic_4[rand_order, :, :, :]

In [ ]:
transferlabel = labels + healthylabels
transferlabel = np.array(transferlabel)
transferlabel = to_categorical(transferlabel)
print(np.shape(transferlabel))
transferlabel = transferlabel[rand_order]

(130, 2)


In [ ]:
pic_1_train = []
pic_1_test = []
pic_1_val = []

pic_2_train = []
pic_2_test = []
pic_2_val = []

pic_3_train = []
pic_3_test = []
pic_3_val = []

pic_4_train = []
pic_4_test = []
pic_4_val = []

trainlabel =[]
testlabel =[]
vallabel = []


for i in range(0,78):
  pic_1_train.append(pic_1[i])
  pic_2_train.append(pic_2[i])
  pic_3_train.append(pic_3[i])
  pic_4_train.append(pic_4[i])
  trainlabel.append(transferlabel[i])



for p in range(78,104):
  pic_1_val.append(pic_1[p])
  pic_2_val.append(pic_2[p])
  pic_3_val.append(pic_3[p])
  pic_4_val.append(pic_4[p])
  vallabel.append(transferlabel[p])


for x in range(104,130):
  pic_1_test.append(pic_1[x])
  pic_2_test.append(pic_2[x])
  pic_3_test.append(pic_3[x])
  pic_4_test.append(pic_4[x])
  testlabel.append(transferlabel[x])




print(np.shape(pic_1_train))
print(np.shape(pic_2_train))
print(np.shape(pic_3_train))
print(np.shape(pic_4_train))

print(np.shape(pic_1_test))
print(np.shape(pic_2_test))
print(np.shape(pic_3_test))
print(np.shape(pic_4_test))

print(np.shape(pic_1_val))
print(np.shape(pic_2_val))
print(np.shape(pic_3_val))
print(np.shape(pic_4_val))

trainlabel = np.array(trainlabel)
testlabel = np.array(testlabel)
vallabel = np.array(vallabel)

(78, 224, 224, 3)
(78, 224, 224, 3)
(78, 224, 224, 3)
(78, 224, 224, 3)
(26, 224, 224, 3)
(26, 224, 224, 3)
(26, 224, 224, 3)
(26, 224, 224, 3)
(26, 224, 224, 3)
(26, 224, 224, 3)
(26, 224, 224, 3)
(26, 224, 224, 3)


In [ ]:
# alexnet
#model = Sequential()
inputs = Input(shape = (desired_size, desired_size, 3))
X = ZeroPadding2D((3, 3))(inputs)
X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
X = BatchNormalization(axis=3, name='bn_conv1')(X)
X = Activation('relu')(X)
X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)
X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')
X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')
X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')
X = X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')
X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)


inputs1 = Input(shape = (desired_size, desired_size, 3))
Y = ZeroPadding2D((3, 3))(inputs1)
Y = Conv2D(64, (7, 7), strides=(2, 2), name='conv2', kernel_initializer=glorot_uniform(seed=0))(Y)
Y = BatchNormalization(axis=3, name='bn_conv2')(Y)
Y = Activation('relu')(Y)
Y = AveragePooling2D(pool_size=(2, 2), padding='same')(Y)
Y = convolutional_block(Y, f=3, filters=[64, 64, 256], stage=2, block='g', s=1)
Y = identity_block(Y, 3, [64, 64, 256], stage=2, block='h')
Y = identity_block(Y, 3, [64, 64, 256], stage=2, block='i')
Y = convolutional_block(Y, f=3, filters=[128, 128, 512], stage=3, block='g', s=2)
Y = identity_block(Y, 3, [128, 128, 512], stage=3, block='h')
Y = identity_block(Y, 3, [128, 128, 512], stage=3, block='i')
Y = identity_block(Y, 3, [128, 128, 512], stage=3, block='j')
Y = convolutional_block(Y, f=3, filters=[256, 256, 1024], stage=4, block='g', s=2)
Y = identity_block(Y, 3, [256, 256, 1024], stage=4, block='h')
Y = identity_block(Y, 3, [256, 256, 1024], stage=4, block='i')
Y = identity_block(Y, 3, [256, 256, 1024], stage=4, block='j')
Y = identity_block(Y, 3, [256, 256, 1024], stage=4, block='k')
Y = identity_block(Y, 3, [256, 256, 1024], stage=4, block='l')
Y = Y = convolutional_block(Y, f=3, filters=[512, 512, 2048], stage=5, block='g', s=2)
Y = identity_block(Y, 3, [512, 512, 2048], stage=5, block='h')
Y = identity_block(Y, 3, [512, 512, 2048], stage=5, block='i')
Y = AveragePooling2D(pool_size=(2, 2), padding='same')(Y)



inputs2 = Input(shape = (desired_size, desired_size, 3))
Z = ZeroPadding2D((3, 3))(inputs2)
Z = Conv2D(64, (7, 7), strides=(2, 2), name='conv3', kernel_initializer=glorot_uniform(seed=0))(Z)
Z = BatchNormalization(axis=3, name='bn_conv3')(Z)
Z = Activation('relu')(Z)
Z = AveragePooling2D(pool_size=(2, 2), padding='same')(Z)
Z = convolutional_block(Z, f=3, filters=[64, 64, 256], stage=2, block='m', s=1)
Z = identity_block(Z, 3, [64, 64, 256], stage=2, block='n')
Z = identity_block(Z, 3, [64, 64, 256], stage=2, block='o')
Z = convolutional_block(Z, f=3, filters=[128, 128, 512], stage=3, block='m', s=2)
Z = identity_block(Z, 3, [128, 128, 512], stage=3, block='n')
Z = identity_block(Z, 3, [128, 128, 512], stage=3, block='o')
Z = identity_block(Z, 3, [128, 128, 512], stage=3, block='p')
Z = convolutional_block(Z, f=3, filters=[256, 256, 1024], stage=4, block='m', s=2)
Z = identity_block(Z, 3, [256, 256, 1024], stage=4, block='n')
Z = identity_block(Z, 3, [256, 256, 1024], stage=4, block='o')
Z = identity_block(Z, 3, [256, 256, 1024], stage=4, block='p')
Z = identity_block(Z, 3, [256, 256, 1024], stage=4, block='q')
Z = identity_block(Z, 3, [256, 256, 1024], stage=4, block='r')
Z = Z = convolutional_block(Z, f=3, filters=[512, 512, 2048], stage=5, block='m', s=2)
Z = identity_block(Z, 3, [512, 512, 2048], stage=5, block='n')
Z = identity_block(Z, 3, [512, 512, 2048], stage=5, block='p')
Z = AveragePooling2D(pool_size=(2, 2), padding='same')(Z)

inputs3 = Input(shape = (desired_size, desired_size, 3))
W = ZeroPadding2D((3, 3))(inputs3)
W = Conv2D(64, (7, 7), strides=(2, 2), name='conv4', kernel_initializer=glorot_uniform(seed=0))(W)
W = BatchNormalization(axis=3, name='bn_conv4')(W)
W = Activation('relu')(W)
W = AveragePooling2D(pool_size=(2, 2), padding='same')(W)
W = convolutional_block(W, f=3, filters=[64, 64, 256], stage=2, block='s', s=1)
W = identity_block(W, 3, [64, 64, 256], stage=2, block='t')
W = identity_block(W, 3, [64, 64, 256], stage=2, block='u')
W = convolutional_block(W, f=3, filters=[128, 128, 512], stage=3, block='s', s=2)
W = identity_block(W, 3, [128, 128, 512], stage=3, block='t')
W = identity_block(W, 3, [128, 128, 512], stage=3, block='u')
W = identity_block(W, 3, [128, 128, 512], stage=3, block='v')
W = convolutional_block(W, f=3, filters=[256, 256, 1024], stage=4, block='s', s=2)
W = identity_block(W, 3, [256, 256, 1024], stage=4, block='t')
W = identity_block(W, 3, [256, 256, 1024], stage=4, block='u')
W = identity_block(W, 3, [256, 256, 1024], stage=4, block='v')
W = identity_block(W, 3, [256, 256, 1024], stage=4, block='w')
W = identity_block(W, 3, [256, 256, 1024], stage=4, block='x')
W = W = convolutional_block(W, f=3, filters=[512, 512, 2048], stage=5, block='s', s=2)
W = identity_block(W, 3, [512, 512, 2048], stage=5, block='t')
W = identity_block(W, 3, [512, 512, 2048], stage=5, block='u')
W = AveragePooling2D(pool_size=(2, 2), padding='same')(W)




concat = Concatenate(axis=-1)([X,Y,Z,W])
add53 = (Conv2D(256, (1, 1), activation='relu', strides=1, padding='same'))(concat)
flattenmax = Flatten()(add53)
dense1 = (Dense(256, activation='relu'))(flattenmax)
dense2 = (Dense(128, activation='relu'))(dense1)
output = (Dense(2, activation='softmax'))(dense2)
model = Model(inputs = [inputs,inputs1,inputs2,inputs3],outputs = output)
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
______________________________________________________________________________________________

In [ ]:
#save weights
checkpoint_filepath_lol = '/content/checkpoint_v1.hdf5'
model_checkpoint_callback = tf.callbacks.ModelCheckpoint(filepath=checkpoint_filepath_lol,save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [ ]:
# complie
optimizer=tf.optimizers.SGD(learning_rate=0.05)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(x=[np.array(pic_1_train),np.array(pic_2_train),np.array(pic_3_train),np.array(pic_4_train)], y=trainlabel, batch_size=32, epochs=50,callbacks=[model_checkpoint_callback],verbose=True,validation_data=([np.array(pic_1_val),np.array(pic_2_val),np.array(pic_3_val),np.array(pic_4_val)],vallabel))

Epoch 1/50
3/3 [==============================] - 42s 3s/step - loss: 10.8346 - accuracy: 0.5128 - val_loss: 4.1849 - val_accuracy: 0.5000
Epoch 2/50
3/3 [==============================] - 2s 613ms/step - loss: 1.4535 - accuracy: 0.6026 - val_loss: 1.1389 - val_accuracy: 0.4615
Epoch 3/50
3/3 [==============================] - 2s 617ms/step - loss: 0.6453 - accuracy: 0.6410 - val_loss: 1.2034 - val_accuracy: 0.5385
Epoch 4/50
3/3 [==============================] - 2s 617ms/step - loss: 0.5627 - accuracy: 0.6923 - val_loss: 1.7390 - val_accuracy: 0.5000
Epoch 5/50
3/3 [==============================] - 2s 620ms/step - loss: 0.5820 - accuracy: 0.5769 - val_loss: 0.8088 - val_accuracy: 0.5769
Epoch 6/50
3/3 [==============================] - 2s 611ms/step - loss: 0.5196 - accuracy: 0.7179 - val_loss: 1.0315 - val_accuracy: 0.5000
Epoch 7/50
3/3 [==============================] - 2s 619ms/step - loss: 0.5011 - accuracy: 0.7051 - val_loss: 0.9832 - val_accuracy: 0.5000
Epoch 8/50
3/3 [=====

In [ ]:
download_CT_COVID = drive.CreateFile({'id':'1bDI0Lokr1Do1adDq6kY2I6NIedaPGDbS'})
download_CT_COVID.GetContentFile('clustercovid.zip')

In [ ]:
!unzip /content/clustercovid.zip -d /content/Data

Archive:  /content/clustercovid.zip
   creating: /content/Data/clustercovid/
  inflating: /content/Data/clustercovid/.DS_Store  
  inflating: /content/Data/__MACOSX/clustercovid/._.DS_Store  
   creating: /content/Data/clustercovid/coviddata/
  inflating: /content/Data/clustercovid/firstcovid.py  
  inflating: /content/Data/__MACOSX/clustercovid/._firstcovid.py  
   creating: /content/Data/clustercovid/coviddata/CT_COVID/
  inflating: /content/Data/__MACOSX/clustercovid/coviddata/._CT_COVID  
  inflating: /content/Data/clustercovid/coviddata/.DS_Store  
  inflating: /content/Data/__MACOSX/clustercovid/coviddata/._.DS_Store  
   creating: /content/Data/clustercovid/coviddata/CT_NonCOVID/
  inflating: /content/Data/__MACOSX/clustercovid/coviddata/._CT_NonCOVID  
  inflating: /content/Data/clustercovid/coviddata/CT_COVID/2020.03.01.20029769-p21-73_1%1.png  
  inflating: /content/Data/__MACOSX/clustercovid/coviddata/CT_COVID/._2020.03.01.20029769-p21-73_1%1.png  
  inflating: /content/Data

In [ ]:
from google.colab import files
file = files.upload()

dfcovidtest = pd.read_csv('update_covid_test.csv')
patientidcovidtest = dfcovidtest["Patient ID"].value_counts().keys().tolist()
countcovidtest = dfcovidtest["Patient ID"].value_counts().tolist()


dfcovidtrain = pd.read_csv('update_covid_train.csv')
patientidcovidtrain = dfcovidtrain["Patient ID"].value_counts().keys().tolist()
countcovidtrain = dfcovidtrain["Patient ID"].value_counts().tolist()


dfcovidval = pd.read_csv('update_covid_val.csv')
patientidcovidval = dfcovidval["Patient ID"].value_counts().keys().tolist()
countcovidval = dfcovidval["Patient ID"].value_counts().tolist()

In [ ]:

from google.colab import files
file = files.upload()

dfnoncovidtest = pd.read_csv('update_noncovid_test.csv')
patientidnoncovidtest = dfnoncovidtest["patient id"].value_counts().keys().tolist()
countnoncovidtest = dfnoncovidtest["patient id"].value_counts().tolist()



dfnoncovidtrain = pd.read_csv('update_noncovid_train.csv')
patientidnoncovidtrain = dfnoncovidtrain["patient id"].value_counts().keys().tolist()
countnoncovidtrain = dfnoncovidtrain["patient id"].value_counts().tolist()



dfnoncovidval = pd.read_csv('update_noncovid_val.csv')
patientidnoncovidval = dfnoncovidval["patient id"].value_counts().keys().tolist()
countnoncovidval = dfnoncovidval["patient id"].value_counts().tolist()

In [ ]:
covidtrainfinalpic = {}
for i in range(inputparameter):
  covidtrainfinalpic['pic'+str(i)] = []
covidtrainfinallabel = []


covidtestfinal = {}
for i in range(inputparameter):
  covidtestfinal['pic'+str(i)] = []
covidtestfinallabel = []

covidvalfinal = {}
for i in range(inputparameter):
  covidvalfinal['pic'+str(i)] = []
covidvalfinallabel = []

In [ ]:
noncovidtrainfinal = {}
for i in range(inputparameter):
  noncovidtrainfinal['nonpic'+str(i)] = []
noncovidtrainfinallabel = []


noncovidtestfinal = {}
for i in range(inputparameter):
  noncovidtestfinal['nonpic'+str(i)] = []
noncovidtestfinallabel = []

noncovidvalfinal = {}
for i in range(inputparameter):
  noncovidvalfinal['nonpic'+str(i)] = []
noncovidvalfinallabel = []

In [ ]:
def createlist(p):
    if (len(p) > inputparameter):
        while (len(p) != inputparameter):
            popvalue = random.randrange(len(p))
            p.pop(popvalue)
    elif (len(p) < inputparameter):
        while (len(p) != inputparameter):
            p.append(np.zeros((desired_size, desired_size, 3)))
    for i in range(len(p)):
        covidtestfinal['pic' + str(i)].append(p[i])


def traincreatelist(p):
    if(len(p) > inputparameter):
        while(len(p) != inputparameter):
            popvalue = random.randrange(len(p))
            p.pop(popvalue)
    elif(len(p) < 8):
        while(len(p) != inputparameter):
            p.append(np.zeros((desired_size, desired_size, 3)))
    for i in range(len(p)):
        covidtrainfinalpic['pic' + str(i)].append(p[i])


def valcreatelist(p):
    if(len(p) > inputparameter):
        while(len(p) != inputparameter):
            popvalue = random.randrange(len(p))
            p.pop(popvalue)
    elif(len(p) < inputparameter):
        while(len(p) != inputparameter):
            p.append(np.zeros((desired_size, desired_size, 3)))
    for i in range(len(p)):
        covidvalfinal['pic' + str(i)].append(p[i])

def myFunc(name):
    temptwo = []
    for i in range(len(name)):
        img = cv2.imread(name[i])
        # print(pId + " iam here" + f1)
        # imgreszie = cv2.resize(img, (480, 480))
        old_size = img.shape[:2]
        ratio = float(desired_size) / max(old_size)
        new_size = tuple([int(x * ratio) for x in old_size])
        img = cv2.resize(img, (new_size[1], new_size[0]))
        delta_w = desired_size - new_size[1]
        delta_h = desired_size - new_size[0]
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)
        color = [0, 0, 0]
        imgreszie = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
        normresize = cv2.normalize(imgreszie, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        #print(normresize)
        temptwo.append(normresize)
    covidtestfinallabel.append(1)
    createlist(temptwo)


def trainmyFunc(name):
    temptwo = []
    for i in range(len(name)):
        img = cv2.imread(name[i])
        # print(pId + " iam here" + f1)
        # imgreszie = cv2.resize(img, (480, 480))
        old_size = img.shape[:2]
        ratio = float(desired_size) / max(old_size)
        new_size = tuple([int(x * ratio) for x in old_size])
        img = cv2.resize(img, (new_size[1], new_size[0]))
        delta_w = desired_size - new_size[1]
        delta_h = desired_size - new_size[0]
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)
        color = [0, 0, 0]
        imgreszie = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
        normresize = cv2.normalize(imgreszie, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        #print(normresize)
        temptwo.append(normresize)
    covidtrainfinallabel.append(1)
    traincreatelist(temptwo)


def valmyFunc(name):
    temptwo = []
    for i in range(len(name)):
        img = cv2.imread(name[i])
        # print(pId + " iam here" + f1)
        # imgreszie = cv2.resize(img, (480, 480))
        old_size = img.shape[:2]
        ratio = float(desired_size) / max(old_size)
        new_size = tuple([int(x * ratio) for x in old_size])
        img = cv2.resize(img, (new_size[1], new_size[0]))
        delta_w = desired_size - new_size[1]
        delta_h = desired_size - new_size[0]
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)
        color = [0, 0, 0]
        imgreszie = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
        normresize = cv2.normalize(imgreszie, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        #print(normresize)
        temptwo.append(normresize)
    covidvalfinallabel.append(1)
    valcreatelist(temptwo)


def seekvalue(y):
    findValue = dfcovidtest.loc[dfcovidtest['Patient ID'] == y]
    templist = []
    for i, row in findValue.iterrows():
        #print(f"Index: {i}")
        image_value = f"{row['File name']}"
        f1 = f'/content/Data/clustercovid/coviddata/CT_COVID/{image_value}'
        #print(f'{findValue} +  " patient id " + {f1}')
        templist.append(f1)
    myFunc(templist)


def trainseekvalue(y):
    findValue = dfcovidtrain.loc[dfcovidtrain['Patient ID'] == y]
    templist = []
    for i, row in findValue.iterrows():
        # print(f"Index: {i}")
        image_value = f"{row['File name']}"
        f1 = f'/content/Data/clustercovid/coviddata/CT_COVID/{image_value}'
        # print(f'{findValue} +  " patient id " + {f1}')
        templist.append(f1)
    trainmyFunc(templist)

def valseekvalue(y):
    findValue = dfcovidval.loc[dfcovidval['Patient ID'] == y]
    templist = []
    for i, row in findValue.iterrows():
        # print(f"Index: {i}")
        image_value = f"{row['File name']}"
        f1 = f'/content/Data/clustercovid/coviddata/CT_COVID/{image_value}'
        # print(f'{findValue} +  " patient id " + {f1}')
        templist.append(f1)
    valmyFunc(templist)



for i in range(len(patientidcovidtest)):
    x = seekvalue(patientidcovidtest[i])


for i in range(len(patientidcovidtrain)):
    x = trainseekvalue(patientidcovidtrain[i])

for i in range(len(patientidcovidval)):
    x = valseekvalue(patientidcovidval[i])

In [ ]:
def noncreatelist(p):
    if (len(p) > inputparameter):
        while (len(p) != inputparameter):
            popvalue = random.randrange(len(p))
            p.pop(popvalue)
    elif (len(p) < inputparameter):
        while (len(p) != inputparameter):
            p.append(np.zeros((desired_size, desired_size, 3)))
    for i in range(len(p)):
        noncovidtestfinal['nonpic' + str(i)].append(p[i])

def nontraincreatelist(p):
    if(len(p) > inputparameter):
        while(len(p) != inputparameter):
            popvalue = random.randrange(len(p))
            p.pop(popvalue)
    elif(len(p) < inputparameter):
        while(len(p) != inputparameter):
            p.append(np.zeros((desired_size, desired_size, 3)))
    for i in range(len(p)):
        noncovidtrainfinal['nonpic' + str(i)].append(p[i])

def nonvalcreatelist(p):
    if(len(p) > inputparameter):
        while(len(p) != inputparameter):
            popvalue = random.randrange(len(p))
            p.pop(popvalue)
    elif(len(p) < inputparameter):
        while(len(p) != inputparameter):
            p.append(np.zeros((desired_size, desired_size, 3)))
    for i in range(len(p)):
        noncovidvalfinal['nonpic' + str(i)].append(p[i])




def nonmyFunc(name):
    temptwo = []
    for i in range(len(name)):
        img = cv2.imread(name[i])
        # print(pId + " iam here" + f1)
        # imgreszie = cv2.resize(img, (480, 480))
        old_size = img.shape[:2]
        ratio = float(desired_size) / max(old_size)
        new_size = tuple([int(x * ratio) for x in old_size])
        img = cv2.resize(img, (new_size[1], new_size[0]))
        delta_w = desired_size - new_size[1]
        delta_h = desired_size - new_size[0]
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)
        color = [0, 0, 0]
        imgreszie = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
        normresize = cv2.normalize(imgreszie, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        #print(normresize)
        temptwo.append(normresize)
    noncovidtestfinallabel.append(0)
    noncreatelist(temptwo)


def nontrainmyFunc(name):
    temptwo = []
    for i in range(len(name)):
        img = cv2.imread(name[i])
        # print(pId + " iam here" + f1)
        # imgreszie = cv2.resize(img, (480, 480))
        old_size = img.shape[:2]
        ratio = float(desired_size) / max(old_size)
        new_size = tuple([int(x * ratio) for x in old_size])
        img = cv2.resize(img, (new_size[1], new_size[0]))
        delta_w = desired_size - new_size[1]
        delta_h = desired_size - new_size[0]
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)
        color = [0, 0, 0]
        imgreszie = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
        normresize = cv2.normalize(imgreszie, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        #print(normresize)
        temptwo.append(normresize)
    noncovidtrainfinallabel.append(0)
    nontraincreatelist(temptwo)


def nonvalmyFunc(name):
    temptwo = []
    for i in range(len(name)):
        img = cv2.imread(name[i])
        # print(pId + " iam here" + f1)
        # imgreszie = cv2.resize(img, (480, 480))
        old_size = img.shape[:2]
        ratio = float(desired_size) / max(old_size)
        new_size = tuple([int(x * ratio) for x in old_size])
        img = cv2.resize(img, (new_size[1], new_size[0]))
        delta_w = desired_size - new_size[1]
        delta_h = desired_size - new_size[0]
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)
        color = [0, 0, 0]
        imgreszie = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
        normresize = cv2.normalize(imgreszie, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        #print(normresize)
        temptwo.append(normresize)
    noncovidvalfinallabel.append(0)
    nonvalcreatelist(temptwo)





def nonseekvalue(y):
    findValue = dfnoncovidtest.loc[dfnoncovidtest['patient id'] == y]
    templist = []
    for i, row in findValue.iterrows():
        #print(f"Index: {i}")
        image_value = f"{row['image name']}"
        f1 = f'/content/Data/clustercovid/coviddata/CT_NonCOVID/{image_value}'
        #print(f'{findValue} +  " patient id " + {f1}')
        templist.append(f1)
    nonmyFunc(templist)


def nontrainseekvalue(y):
    findValue = dfnoncovidtrain.loc[dfnoncovidtrain['patient id'] == y]
    templist = []
    for i, row in findValue.iterrows():
        # print(f"Index: {i}")
        image_value = f"{row['image name']}"
        f1 = f'/content/Data/clustercovid/coviddata/CT_NonCOVID/{image_value}'
        # print(f'{findValue} +  " patient id " + {f1}')
        templist.append(f1)
    nontrainmyFunc(templist)

def nonvalseekvalue(y):
    findValue = dfnoncovidval.loc[dfnoncovidval['patient id'] == y]
    templist = []
    for i, row in findValue.iterrows():
        # print(f"Index: {i}")
        image_value = f"{row['image name']}"
        f1 = f'/content/Data/clustercovid/coviddata/CT_NonCOVID/{image_value}'
        # print(f'{findValue} +  " patient id " + {f1}')
        templist.append(f1)
    nonvalmyFunc(templist)


for i in range(len(patientidnoncovidtest)):
    x = nonseekvalue(patientidnoncovidtest[i])

for i in range(len(patientidnoncovidtrain)):
    x = nontrainseekvalue(patientidnoncovidtrain[i])

for i in range(len(patientidnoncovidval)):
    x = nonvalseekvalue(patientidnoncovidval[i])

In [ ]:
endtrainlabel = covidtrainfinallabel + noncovidtrainfinallabel
endtrainlabel = np.array(endtrainlabel)
endtrainlabel = to_categorical(endtrainlabel)
#endtrainlabel = np.reshape(endtrainlabel,(1,233,2))


endtestlabel = covidtestfinallabel + noncovidtestfinallabel
endtestlabel = np.array(endtestlabel)
endtestlabel = to_categorical(endtestlabel)
#endtestlabel = np.reshape(endtestlabel,(1,95,2))

endvalabel = covidvalfinallabel + noncovidvalfinallabel
endvalabel = np.array(endvalabel)
endvalabel = to_categorical(endvalabel)
#endvalabel = np.reshape(endvalabel,(1,56,2))

In [ ]:
print(np.shape(endtrainlabel))
print(np.shape(endtestlabel))
print(np.shape(endvalabel))



In [ ]:
pic1 = (list(covidtrainfinalpic.values())[0] + list(noncovidtrainfinal.values())[0])
pic2 = (list(covidtrainfinalpic.values())[1] + list(noncovidtrainfinal.values())[1])
pic3 = (list(covidtrainfinalpic.values())[2] + list(noncovidtrainfinal.values())[2])
pic4 = (list(covidtrainfinalpic.values())[3] + list(noncovidtrainfinal.values())[3])


pic11 = (list(covidtestfinal.values())[0] + list(noncovidtestfinal.values())[0])
pic22 = (list(covidtestfinal.values())[1] + list(noncovidtestfinal.values())[1])
pic33 = (list(covidtestfinal.values())[2] + list(noncovidtestfinal.values())[2])
pic44 = (list(covidtestfinal.values())[3] + list(noncovidtestfinal.values())[3])


pic111 = (list(covidvalfinal.values())[0] + list(noncovidvalfinal.values())[0])
pic222 = (list(covidvalfinal.values())[1] + list(noncovidvalfinal.values())[1])
pic333 = (list(covidvalfinal.values())[2] + list(noncovidvalfinal.values())[2])
pic444 = (list(covidvalfinal.values())[3] + list(noncovidvalfinal.values())[3])

In [ ]:
# alexnet
#model = Sequential()
inputs = Input(shape = (desired_size, desired_size, 3))
X = ZeroPadding2D((3, 3))(inputs)
X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
X = BatchNormalization(axis=3, name='bn_conv1')(X)
X = Activation('relu')(X)
X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)
X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')
X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')
X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')
X = X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')
X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)


inputs1 = Input(shape = (desired_size, desired_size, 3))
Y = ZeroPadding2D((3, 3))(inputs1)
Y = Conv2D(64, (7, 7), strides=(2, 2), name='conv2', kernel_initializer=glorot_uniform(seed=0))(Y)
Y = BatchNormalization(axis=3, name='bn_conv2')(Y)
Y = Activation('relu')(Y)
Y = AveragePooling2D(pool_size=(2, 2), padding='same')(Y)
Y = convolutional_block(Y, f=3, filters=[64, 64, 256], stage=2, block='g', s=1)
Y = identity_block(Y, 3, [64, 64, 256], stage=2, block='h')
Y = identity_block(Y, 3, [64, 64, 256], stage=2, block='i')
Y = convolutional_block(Y, f=3, filters=[128, 128, 512], stage=3, block='g', s=2)
Y = identity_block(Y, 3, [128, 128, 512], stage=3, block='h')
Y = identity_block(Y, 3, [128, 128, 512], stage=3, block='i')
Y = identity_block(Y, 3, [128, 128, 512], stage=3, block='j')
Y = convolutional_block(Y, f=3, filters=[256, 256, 1024], stage=4, block='g', s=2)
Y = identity_block(Y, 3, [256, 256, 1024], stage=4, block='h')
Y = identity_block(Y, 3, [256, 256, 1024], stage=4, block='i')
Y = identity_block(Y, 3, [256, 256, 1024], stage=4, block='j')
Y = identity_block(Y, 3, [256, 256, 1024], stage=4, block='k')
Y = identity_block(Y, 3, [256, 256, 1024], stage=4, block='l')
Y = Y = convolutional_block(Y, f=3, filters=[512, 512, 2048], stage=5, block='g', s=2)
Y = identity_block(Y, 3, [512, 512, 2048], stage=5, block='h')
Y = identity_block(Y, 3, [512, 512, 2048], stage=5, block='i')
Y = AveragePooling2D(pool_size=(2, 2), padding='same')(Y)



inputs2 = Input(shape = (desired_size, desired_size, 3))
Z = ZeroPadding2D((3, 3))(inputs2)
Z = Conv2D(64, (7, 7), strides=(2, 2), name='conv3', kernel_initializer=glorot_uniform(seed=0))(Z)
Z = BatchNormalization(axis=3, name='bn_conv3')(Z)
Z = Activation('relu')(Z)
Z = AveragePooling2D(pool_size=(2, 2), padding='same')(Z)
Z = convolutional_block(Z, f=3, filters=[64, 64, 256], stage=2, block='m', s=1)
Z = identity_block(Z, 3, [64, 64, 256], stage=2, block='n')
Z = identity_block(Z, 3, [64, 64, 256], stage=2, block='o')
Z = convolutional_block(Z, f=3, filters=[128, 128, 512], stage=3, block='m', s=2)
Z = identity_block(Z, 3, [128, 128, 512], stage=3, block='n')
Z = identity_block(Z, 3, [128, 128, 512], stage=3, block='o')
Z = identity_block(Z, 3, [128, 128, 512], stage=3, block='p')
Z = convolutional_block(Z, f=3, filters=[256, 256, 1024], stage=4, block='m', s=2)
Z = identity_block(Z, 3, [256, 256, 1024], stage=4, block='n')
Z = identity_block(Z, 3, [256, 256, 1024], stage=4, block='o')
Z = identity_block(Z, 3, [256, 256, 1024], stage=4, block='p')
Z = identity_block(Z, 3, [256, 256, 1024], stage=4, block='q')
Z = identity_block(Z, 3, [256, 256, 1024], stage=4, block='r')
Z = Z = convolutional_block(Z, f=3, filters=[512, 512, 2048], stage=5, block='m', s=2)
Z = identity_block(Z, 3, [512, 512, 2048], stage=5, block='n')
Z = identity_block(Z, 3, [512, 512, 2048], stage=5, block='p')
Z = AveragePooling2D(pool_size=(2, 2), padding='same')(Z)

inputs3 = Input(shape = (desired_size, desired_size, 3))
W = ZeroPadding2D((3, 3))(inputs3)
W = Conv2D(64, (7, 7), strides=(2, 2), name='conv4', kernel_initializer=glorot_uniform(seed=0))(W)
W = BatchNormalization(axis=3, name='bn_conv4')(W)
W = Activation('relu')(W)
W = AveragePooling2D(pool_size=(2, 2), padding='same')(W)
W = convolutional_block(W, f=3, filters=[64, 64, 256], stage=2, block='s', s=1)
W = identity_block(W, 3, [64, 64, 256], stage=2, block='t')
W = identity_block(W, 3, [64, 64, 256], stage=2, block='u')
W = convolutional_block(W, f=3, filters=[128, 128, 512], stage=3, block='s', s=2)
W = identity_block(W, 3, [128, 128, 512], stage=3, block='t')
W = identity_block(W, 3, [128, 128, 512], stage=3, block='u')
W = identity_block(W, 3, [128, 128, 512], stage=3, block='v')
W = convolutional_block(W, f=3, filters=[256, 256, 1024], stage=4, block='s', s=2)
W = identity_block(W, 3, [256, 256, 1024], stage=4, block='t')
W = identity_block(W, 3, [256, 256, 1024], stage=4, block='u')
W = identity_block(W, 3, [256, 256, 1024], stage=4, block='v')
W = identity_block(W, 3, [256, 256, 1024], stage=4, block='w')
W = identity_block(W, 3, [256, 256, 1024], stage=4, block='x')
W = W = convolutional_block(W, f=3, filters=[512, 512, 2048], stage=5, block='s', s=2)
W = identity_block(W, 3, [512, 512, 2048], stage=5, block='t')
W = identity_block(W, 3, [512, 512, 2048], stage=5, block='u')
W = AveragePooling2D(pool_size=(2, 2), padding='same')(W)




concat = Concatenate(axis=-1)([X,Y,Z,W])
add53 = (Conv2D(256, (1, 1), activation='relu', strides=1, padding='same'))(concat)
flattenmax = Flatten()(add53)
dense1 = (Dense(256, activation='relu'))(flattenmax)
dense2 = (Dense(128, activation='relu'))(dense1)
output = (Dense(2, activation='softmax'))(dense2)
model = Model(inputs = [inputs,inputs1,inputs2,inputs3],outputs = output)
model.summary()


In [ ]:
#save weights
checkpoint_filepath = '/content/checkpoint.hdf5'
model_checkpoint_callback = tf.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = (true_positives * 1.0) / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = (true_positives * 1.0) / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
# complie
optimizer=tf.optimizers.SGD(learning_rate=0.05)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy',f1_m,precision_m, recall_m])

In [ ]:
model.load_weights(checkpoint_filepath_lol)
history = model.fit(x=[np.array(pic1),np.array(pic2),np.array(pic3),np.array(pic4)], y=endtrainlabel, batch_size=32, epochs=50,callbacks=[model_checkpoint_callback],verbose=True,validation_data=([np.array(pic111),np.array(pic222),np.array(pic333),np.array(pic444)],endvalabel))


model.load_weights(checkpoint_filepath)

loss, accuracy, f1_score, precision, recall = model.evaluate([np.array(pic11),np.array(pic22),np.array(pic33),np.array(pic44)],endtestlabel,verbose=1)
print(f'Test loss: {loss:.3}')
print(f'Test accuracy: {accuracy:.3}')
print(f'Test f1_score: {f1_score:.3}')
print(f'Test precision: {precision:.3}')
print(f'Test recall: {recall:.3}')
